In [84]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from model import Agent
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [85]:
df = pd.read_csv('clean_health_data.csv')
df

,Billing_Amount,Age,admited_days,Gender_Male,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,Blood Type_B-,Blood Type_O+,...,Insurance Provider_Medicare,Insurance Provider_UnitedHealthcare,Admission Type_Emergency,Admission Type_Urgent,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Test Results_Inconclusive,Test Results_Normal
0,18856.281306,0.223684,0.034483,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
1,33643.327287,0.644737,0.172414,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
2,27955.096079,0.828947,0.482759,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,37909.782410,0.197368,1.000000,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
4,14238.317814,0.394737,0.655172,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,2650.714952,0.381579,1.000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
55496,31457.797307,0.631579,0.275862,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55497,27620.764717,0.328947,0.931034,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
55498,32451.092358,0.394737,0.172414,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [86]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.1, random_state =43 )
Y_train

18608    48735.848001
20123    42615.243851
9749      3995.424761
10325    26608.430185
27068    28733.379588
             ...     
18448    30896.731703
40753    28559.161643
51455    24378.326343
19776    29357.089704
14148    33211.426429
Name: Billing_Amount, Length: 49950, dtype: float64

In [41]:
def scale_to_minus1_1(X):
    X_min = np.min(X)
    X_max = np.max(X)
    return -1 + 2 * (X - X_min) / (X_max - X_min)

In [43]:
Y_train = scale_to_minus1_1(Y_train)
Y_train

18608    0.852904
20123    0.629413
9749    -0.780770
10325    0.044932
27068    0.122524
           ...   
18448    0.201518
40753    0.116162
51455   -0.036499
19776    0.145298
14148    0.286038
Name: Billing_Amount, Length: 49950, dtype: float64

In [131]:
agent = Agent(X.shape[1], 1)

In [132]:
def calculate_reward(pred, label):
    pred = np.array(pred)
    label = np.array(label)
    return - np.mean((pred-label)**2)

In [133]:
total_rewards = []
episodes = 2000
batch_size = 32

for episode in range(episodes):
    loss = 0 
    for i in range(0, len(X_train), batch_size):
        end = i + batch_size
        states = X_train[i:end]
        states = np.expand_dims(states, axis=2).astype('float32')
        labels = Y_train[i:end]

        actions = agent.get_action(states)
        
        reward = calculate_reward(labels,actions)

        next_states = states

        data = [states, actions, states, reward]
        loss += np.mean(reward)
        agent.train(data)

    if episode % 2 == 0:
        agent.update()
        print("Update!")

    print(f'Epoch {episode}, Average Loss: {-loss / len(X_train)}')



Update!
Epoch 0, Average Loss: 26745849.6524424
Epoch 1, Average Loss: 26746511.894430306
Update!
Epoch 2, Average Loss: 26745695.109493893
Epoch 3, Average Loss: 26745808.424969047
Update!
Epoch 4, Average Loss: 26746198.33321694


KeyboardInterrupt: 